In [1]:
import psycopg2 as db

In [2]:
conn_string = "dbname='postgres' host='localhost' user='postgres' password='Tuan1211'"

In [3]:
conn = db.connect(conn_string)

In [4]:
cur = conn.cursor()

In [5]:
query = " insert into users (id, name, street, city, zip) values('{}','{}', '{}','{}', '{}')".format(1, 'Big Bird', 'Sesame Street', 'Fakeville', '12345')

In [6]:
print(cur.mogrify(query))

b" insert into users (id, name, street, city, zip) values('1','Big Bird', 'Sesame Street','Fakeville', '12345')"


In [10]:
query2 = "insert into users (id,name,street,city,zip) values(%s,%s,%s,%s,%s)"
data=(1,'Big Bird','Sesame Street','Fakeville','12345')
#print(cur.mogrify(query2,data))
cur.execute(query2,data)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
conn.commit()